In [2]:
#!pip list
!pip install pandas scikit-learn matplotlib seaborn librosa

  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached msgpack-1.1.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 912.1 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 800.5 kB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 454.5 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 460.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 461.1 kB/s eta 0:00:00a 0:00:01
Using cached msgpack-1.1.0-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (399 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 578.2 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 329.3 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 390.6 kB/s eta 0:00:00a 0:00:01
Using cached pytz-2025.2-py2

In [1]:
import pandas as pd
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns
import ast

In [2]:
# Load raw data
df = pd.read_csv('train_set.csv')


In [3]:
df.head()

,file,sampling_rate,channel_1,channel_2,ratios,label
0,8ab4ffda-6f7d-4fd8-9fc4-5d00e7798d43_22281325_...,8000,{'segments': {'segment_1': {'f0_mean': 329.527...,{'segments': {'segment_1': {'f0_mean': 465.244...,"{'overall': {'f0_mean': 0.6529480783025756, 'f...",1
1,fec28b84-69e8-4d83-bea1-70d825050057_22237809_...,8000,{'segments': {'segment_1': {'f0_mean': 369.308...,{'segments': {'segment_1': {'f0_mean': 434.658...,"{'overall': {'f0_mean': 0.6186414294558663, 'f...",0
2,007554e2-9c6f-495b-8a9e-b0afbd948bf3_22197563_...,8000,{'segments': {'segment_1': {'f0_mean': 298.181...,{'segments': {'segment_1': {'f0_mean': 402.850...,"{'overall': {'f0_mean': 0.7379481569593161, 'f...",0
3,2d4b31ef-a683-4d9e-bc54-3d7bd5a758d8_22237750_...,8000,{'segments': {'segment_1': {'f0_mean': 356.413...,{'segments': {'segment_1': {'f0_mean': 395.966...,"{'overall': {'f0_mean': 0.9128097966131058, 'f...",0
4,b9a6592a-376e-4094-8ab5-e7dd25fac4c1_22214911_...,8000,{'segments': {'segment_1': {'f0_mean': 355.541...,{'segments': {'segment_1': {'f0_mean': 371.768...,"{'overall': {'f0_mean': 0.9219474432548425, 'f...",0


In [4]:
df.shape

(895, 6)